In [2]:
import torch
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston

In [3]:
bos = load_boston()
print(bos.keys())
print('shape:',bos.data.shape)


dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])
shape: (506, 13)


## Preprocessing
We load our data to a dataframe in Pandas.

In [4]:
df = pd.DataFrame(bos.data)
df.columns = bos.feature_names
df['Price'] = bos.target
df.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,Price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
X_unNormed = df.drop('Price', axis=1).to_numpy()
Y_unNormed = df['Price'].to_numpy()



In [6]:
def normalise_rows(x):
    x_norm = np.linalg.norm(x)
    x = x/x_norm
    return x


In [7]:
X = normalise_rows(X_unNormed)
Y = normalise_rows(Y_unNormed)

In [16]:
np.reshape?

In [90]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.3, random_state = 42)

print(X_train.shape)
print(X_test.shape)


Y_train = Y_train.reshape((354,1))
Y_test = Y_test.reshape((152, 1))

print(Y_train.shape)
print(Y_test.shape)


(354, 13)
(152, 13)
(354, 1)
(152, 1)


In [91]:
inputs = torch.from_numpy(X_train).float()
targets = torch.from_numpy(Y_train).float()
print('shape_input:',inputs.shape)
print('shape_targets:', targets.shape)


shape_input: torch.Size([354, 13])
shape_targets: torch.Size([354, 1])


In [92]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader


In [93]:
train_ds = TensorDataset(inputs,targets)
print(train_ds)

In [95]:
batch_size = 5
train_d1 = DataLoader (train_ds, batch_size, shuffle = True)

In [96]:
for xb, yb in train_d1:
    print(xb)
    print(yb)
    break

tensor([[5.0230e-05, 1.5299e-03, 3.0368e-04, 0.0000e+00, 4.9492e-05, 5.2337e-04,
         7.6494e-03, 1.5381e-04, 3.8247e-04, 2.0195e-02, 9.9443e-04, 2.9980e-02,
         5.2781e-04],
        [3.1188e-05, 0.0000e+00, 4.7426e-04, 7.6494e-05, 3.8783e-05, 4.7151e-04,
         6.9839e-03, 2.3315e-04, 6.1195e-04, 2.3484e-02, 1.3310e-03, 3.0234e-02,
         1.6416e-03],
        [6.6275e-06, 3.4422e-03, 2.6314e-04, 0.0000e+00, 3.3428e-05, 5.4908e-04,
         2.0118e-03, 4.9567e-04, 3.8247e-04, 3.0445e-02, 1.1627e-03, 2.9870e-02,
         2.1954e-04],
        [1.3692e-05, 0.0000e+00, 7.4123e-04, 0.0000e+00, 4.4749e-05, 4.3372e-04,
         2.2030e-03, 2.1408e-04, 4.5897e-04, 2.9909e-02, 1.4687e-03, 3.0084e-02,
         1.3463e-03],
        [2.9232e-05, 0.0000e+00, 4.7426e-04, 0.0000e+00, 3.8553e-05, 6.1501e-04,
         6.6168e-03, 2.4598e-04, 6.1195e-04, 2.3484e-02, 1.3310e-03, 2.9632e-02,
         2.3943e-04]])
tensor([[0.0550],
        [0.0396],
        [0.0665],
        [0.0422],
       

In [97]:
import torch.nn as nn

In [98]:
model = nn.Linear(13,1)

In [99]:
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1455,  0.2655, -0.1234, -0.1785, -0.0076,  0.1355,  0.2550,  0.0909,
         -0.1266, -0.0304, -0.2361,  0.0246,  0.2114]], requires_grad=True)
Parameter containing:
tensor([0.0026], requires_grad=True)


In [100]:
preds = model(inputs.float())


## Loss function

In [101]:
import torch.nn.functional as F

In [108]:
loss_fn = F.mse_loss
loss = loss_fn
print(loss)

<function mse_loss at 0x7f7829485710>


## Optimizer


In [109]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [121]:
def fit(num_epochs, model, loss_fn, opt, train_d1):
    # repeat for number of epochs
    for epoch in range(num_epochs):
        # train with batches of data
        for xb, yb in train_d1:
            # 1. generate predictions
            pred = model(xb)
            # 2. calculate loss
            loss = loss_fn(pred, yb)
            # 3. compute gradients
            loss.backward()
            # 4. update parameters using gradients
            opt.step()
            # 5. reset gradients to zero
            opt.zero_grad()
        if (epoch+1) % 100 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            
            

In [122]:
fit(10000, model, loss_fn, opt, train_d1)

Epoch [100/10000], Loss: 0.0002
Epoch [200/10000], Loss: 0.0001
Epoch [300/10000], Loss: 0.0005
Epoch [400/10000], Loss: 0.0003
Epoch [500/10000], Loss: 0.0002
Epoch [600/10000], Loss: 0.0001
Epoch [700/10000], Loss: 0.0001
Epoch [800/10000], Loss: 0.0000
Epoch [900/10000], Loss: 0.0000
Epoch [1000/10000], Loss: 0.0000
Epoch [1100/10000], Loss: 0.0002
Epoch [1200/10000], Loss: 0.0000
Epoch [1300/10000], Loss: 0.0002
Epoch [1400/10000], Loss: 0.0010
Epoch [1500/10000], Loss: 0.0001
Epoch [1600/10000], Loss: 0.0002
Epoch [1700/10000], Loss: 0.0001
Epoch [1800/10000], Loss: 0.0001
Epoch [1900/10000], Loss: 0.0001
Epoch [2000/10000], Loss: 0.0001
Epoch [2100/10000], Loss: 0.0009
Epoch [2200/10000], Loss: 0.0000
Epoch [2300/10000], Loss: 0.0003
Epoch [2400/10000], Loss: 0.0001
Epoch [2500/10000], Loss: 0.0001
Epoch [2600/10000], Loss: 0.0001
Epoch [2700/10000], Loss: 0.0011
Epoch [2800/10000], Loss: 0.0004
Epoch [2900/10000], Loss: 0.0000
Epoch [3000/10000], Loss: 0.0006
Epoch [3100/10000],

In [123]:
preds = model(inputs.float())

In [124]:
preds

tensor([[0.0421],
        [0.0419],
        [0.0417],
        [0.0412],
        [0.0412],
        [0.0413],
        [0.0418],
        [0.0425],
        [0.0425],
        [0.0427],
        [0.0416],
        [0.0418],
        [0.0426],
        [0.0425],
        [0.0431],
        [0.0417],
        [0.0425],
        [0.0431],
        [0.0413],
        [0.0425],
        [0.0416],
        [0.0423],
        [0.0412],
        [0.0418],
        [0.0421],
        [0.0414],
        [0.0420],
        [0.0425],
        [0.0427],
        [0.0421],
        [0.0419],
        [0.0417],
        [0.0422],
        [0.0425],
        [0.0408],
        [0.0426],
        [0.0414],
        [0.0414],
        [0.0414],
        [0.0422],
        [0.0420],
        [0.0417],
        [0.0421],
        [0.0424],
        [0.0415],
        [0.0424],
        [0.0416],
        [0.0423],
        [0.0424],
        [0.0421],
        [0.0423],
        [0.0424],
        [0.0422],
        [0.0421],
        [0.0420],
        [0

In [125]:
targets

tensor([[0.0524],
        [0.0387],
        [0.0353],
        [0.0422],
        [0.0349],
        [0.0457],
        [0.0610],
        [0.0091],
        [0.0541],
        [0.0342],
        [0.0396],
        [0.0422],
        [0.0417],
        [0.0384],
        [0.0892],
        [0.0267],
        [0.0303],
        [0.0495],
        [0.0367],
        [0.0362],
        [0.0384],
        [0.0755],
        [0.0424],
        [0.0373],
        [0.0338],
        [0.0537],
        [0.0665],
        [0.0446],
        [0.0216],
        [0.0252],
        [0.0225],
        [0.0325],
        [0.0605],
        [0.0488],
        [0.0245],
        [0.0263],
        [0.0913],
        [0.0402],
        [0.0364],
        [0.0435],
        [0.0320],
        [0.0232],
        [0.0102],
        [0.0568],
        [0.0479],
        [0.0354],
        [0.0305],
        [0.0252],
        [0.0418],
        [0.0280],
        [0.0502],
        [0.0660],
        [0.0418],
        [0.0448],
        [0.0457],
        [0